#Downloads

In [14]:
%pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 7.43 s (started: 2023-11-28 11:06:31 +00:00)


In [15]:
!pip install polars icecream scipy

time: 5.03 s (started: 2023-11-28 11:06:39 +00:00)


#Imports

In [16]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split, Dataset
# import gdown
import gzip
%load_ext autotime
import pandas as pd
from sklearn.model_selection import train_test_split
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import torch.multiprocessing as mp
from torchsummary import summary
import polars as pl
import math as m
import numpy as np
from icecream import ic
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
import seaborn as sns
import sklearn
import random

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.6 ms (started: 2023-11-28 11:06:44 +00:00)


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

time: 3.4 ms (started: 2023-11-28 11:06:44 +00:00)


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 4.08 s (started: 2023-11-28 11:06:44 +00:00)


# Pickle  function

In [19]:
path = r"/content/drive/MyDrive/Colab Notebooks/FML_Project/"
def pickleThis(file,pklname):
    with open(path+pklname+".pkl", 'wb') as f:
        pickle.dump(file, f)

def loadPickle(filename):
    with open(path+filename, 'rb') as f:
        return pickle.load(f)

time: 794 µs (started: 2023-11-28 11:06:48 +00:00)


#Load the data

- Creating pairs
- Dataloader

In [20]:
x_train = loadPickle("x_train.pkl")
x_val = loadPickle("x_val.pkl")

y_train = loadPickle("y_train.pkl")
y_val = loadPickle("y_val.pkl")

time: 20.4 s (started: 2023-11-28 11:06:49 +00:00)


In [21]:
random.seed(42)

num_train_elements = len(x_train)
num_val_elements = len(x_val)

#random index orders for both train and val
train_index_order = random.sample(range(num_train_elements), num_train_elements)
val_index_order = random.sample(range(num_val_elements), num_val_elements)

#shuffle data and labels for both train and val based on the random index orders
x_train = np.array([x_train[i] for i in train_index_order])
y_train = np.array([y_train[i] for i in train_index_order])

x_val = np.array([x_val[i] for i in val_index_order])
y_val = np.array([y_val[i] for i in val_index_order])

time: 1.85 s (started: 2023-11-28 11:07:37 +00:00)


##Create pairs

In [22]:
def create_pairs(FFT_inputs, labels):
  InputPairs = []
  labelPairs = []


  fault_classes = 3
  #Getting the indices of each class
  indices = [np.where(labels ==i)[0] for i in range(fault_classes)]

  for index in range(len(FFT_inputs)):
    #Getting current input with index
    Xi = FFT_inputs[index]
    #getting the label of xi.
    label = labels[index]

    #Randomly choosing another labels from the same class
    indB = np.random.choice(indices[label])
    #corresponding Xj for this randomly selected label
    Xj = FFT_inputs[indB]

    InputPairs.append([Xi, Xj])
    labelPairs.append([1])

    #Getting a label where label is different than the current xi
    diss_index = np.where(labels != label)[0]

    #finding an image for this label
    diss_Xj = FFT_inputs[np.random.choice(diss_index)]
    InputPairs.append([Xi, diss_Xj])
    labelPairs.append([0])

  return (np.array(InputPairs), np.array(labelPairs))

time: 671 µs (started: 2023-11-28 11:07:39 +00:00)


In [23]:
(x_trainpair, y_trainpair) = create_pairs(x_train, y_train)
(x_valpair, y_valpair) = create_pairs(x_val, y_val)

time: 887 ms (started: 2023-11-28 11:07:41 +00:00)


In [24]:
print(x_trainpair.shape,x_valpair.shape)

(9420, 2, 1, 10451) (4060, 2, 1, 10451)
time: 2.39 ms (started: 2023-11-28 11:07:45 +00:00)


##Convert to tensors and data-loader

In [ ]:
batch_size = 1028
class TransformDataset(Dataset):
    def __init__(self, pairs, labels):
        self.pairs = pairs
        self.labels = labels

    def __getitem__(self, index):
        pair = self.pairs[index]
        label = self.labels[index]
        return torch.tensor(pair[0], dtype=torch.float32), torch.tensor(pair[1], dtype=torch.float32), label

    def __len__(self):
        return len(self.labels)

def create_dataloader(pairs, labels, batch_size):
    dataset = TransformDataset(pairs, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

train_dataloader = create_dataloader(x_trainpair, y_trainpair, batch_size)
val_dataloader = create_dataloader(x_valpair, y_valpair, batch_size)

time: 1.6 ms (started: 2023-11-28 06:45:27 +00:00)


In [ ]:
sample_batch = next(iter(TransformDataset(x_trainpair, y_trainpair)))
print("Sample Batch from Siamese Dataset:")
for i, data in enumerate(sample_batch):
    print(f"Data {i} shape: {data.shape}")


dataiter = iter(train_dataloader)
input1, input2, label = next(dataiter)

print("\nBatch from Siamese Data Loader:")
print("Input 1:")
print(f"Input 1 shape: {input1.shape}")
print("\nInput 2:")
print(f"Input 2 shape: {input2.shape}")

print("\nLabels:")
print(f"Labels shape: {label.shape}")

Sample Batch from Siamese Dataset:
Data 0 shape: torch.Size([1, 10451])
Data 1 shape: torch.Size([1, 10451])
Data 2 shape: (1,)

Batch from Siamese Data Loader:
Input 1:
Input 1 shape: torch.Size([1028, 1, 10451])

Input 2:
Input 2 shape: torch.Size([1028, 1, 10451])

Labels:
Labels shape: torch.Size([1028, 1])
time: 132 ms (started: 2023-11-28 06:45:29 +00:00)


#SNN architecture

In [8]:
class SNN(nn.Module):
    def __init__(self):
        super(SNN, self).__init__()

        self.cnn1 = nn.Sequential(


            nn.Conv1d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding="same"),
            nn.BatchNorm1d(8),
            nn.ReLU(inplace=True),
            nn.AvgPool1d(2),


            nn.Conv1d(in_channels=8, out_channels=12, kernel_size=3, stride=1, padding="same"),
            nn.BatchNorm1d(12),
            nn.ReLU(inplace=True),
            nn.AvgPool1d(2),
        )

        #the fully connected layers
        self.fc1 = nn.Sequential(
            nn.Linear(2612 * 12, 1024),
            nn.ReLU(inplace=True),

            nn.Linear(1024, 128),
            nn.ReLU(inplace=True),

            nn.Linear(128,2),
            )


    def forward_once(self, x):

        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        # forward pass of input 1
        output1 = self.forward_once(input1)
        # forward pass of input 2
        output2 = self.forward_once(input2)
        return output1, output2

time: 912 µs (started: 2023-11-28 11:04:22 +00:00)


In [ ]:
# Create an instance of the model
model = SNN()
model = model.to(device)
input1 = input1.to(device)
input2 = input2.to(device)
summary(model, [(1, 10451), (1, 10451)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 8, 10451]              32
       BatchNorm1d-2             [-1, 8, 10451]              16
              ReLU-3             [-1, 8, 10451]               0
         AvgPool1d-4              [-1, 8, 5225]               0
            Conv1d-5             [-1, 12, 5225]             300
       BatchNorm1d-6             [-1, 12, 5225]              24
              ReLU-7             [-1, 12, 5225]               0
         AvgPool1d-8             [-1, 12, 2612]               0
            Linear-9                 [-1, 1024]      32,097,280
             ReLU-10                 [-1, 1024]               0
           Linear-11                  [-1, 128]         131,200
             ReLU-12                  [-1, 128]               0
           Linear-13                    [-1, 2]             258
           Conv1d-14             [-1, 8

###Contrastive loss

In [ ]:
class ContrastiveLoss(nn.Module):
    "Contrastive loss function"
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean(
            (1-label) * torch.pow(euclidean_distance, 2)+ (label)* torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

time: 663 µs (started: 2023-11-28 06:45:48 +00:00)


In [ ]:
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0004)

time: 1 ms (started: 2023-11-28 06:45:50 +00:00)


### Training

In [ ]:
def train(train_dataloader):
    loss = []
    counter = []

    with tqdm(total=len(train_dataloader), desc=f"Epoch {epoch} - Training", unit="batch") as pbar:
        for i, data in enumerate(train_dataloader, 0):
            x1, x2, label = data
            x1, x2, label = x1.cuda(), x2.cuda(), label.cuda()
            optimizer.zero_grad()
            output1, output2 = model(x1, x2)
            loss_contrastive = criterion(output1, output2, label)
            loss_contrastive.backward()
            optimizer.step()
            loss.append(loss_contrastive.item())

            # Update the progress bar
            pbar.update(1)
    loss = np.array(loss)
    return loss.mean() / len(train_dataloader)

def eval(eval_dataloader):
    loss = []

    with tqdm(total=len(eval_dataloader)) as pbar:
        for i, data in enumerate(eval_dataloader, 0):
            x1, x2, label = data
            x1, x2, label = x1.cuda(), x2.cuda(), label.cuda()
            output1, output2 = model(x1, x2)

            loss_contrastive = criterion(output1, output2, label)
            loss.append(loss_contrastive.item())
            pbar.update(1)

    loss = np.array(loss)
    return loss.mean() / len(eval_dataloader)

time: 3.2 ms (started: 2023-11-28 06:48:39 +00:00)


### Training

In [ ]:
for epoch in range(1, 10):
    train_loss = train(train_dataloader)
    eval_loss = eval(val_dataloader)
    print("-" * 50)
    print(f"Training loss: {train_loss}")
    print("-" * 50)
    print(f"\nEval loss: {eval_loss}")

100%|██████████| 4/4 [00:00<00:00,  4.50it/s]


--------------------------------------------------
Training loss: 0.03859498649835587
--------------------------------------------------
Eval loss: 0.07650486752390862


100%|██████████| 4/4 [00:00<00:00,  4.53it/s]


--------------------------------------------------
Training loss: 0.029792310893535612
--------------------------------------------------
Eval loss: 0.07551160268485546


100%|██████████| 4/4 [00:01<00:00,  3.73it/s]


...

--------------------------------------------------
Training loss: 0.028881066739559175
--------------------------------------------------
Eval loss: 0.07539464719593525


100%|██████████| 4/4 [00:00<00:00,  4.41it/s]


--------------------------------------------------
Training loss: 0.028919283151626583
--------------------------------------------------
Eval loss: 0.07562603428959846


100%|██████████| 4/4 [00:01<00:00,  3.25it/s]

--------------------------------------------------
Training loss: 0.02895043581724167
--------------------------------------------------
Eval loss: 0.07527168281376362
time: 49.6 s (started: 2023-11-28 06:48:54 +00:00)


#Export and load model

In [ ]:
# Export
torch.save(model, path+"/model.pth")

#Demo

In [13]:
# # Load the data
# df = pl.read_ipc(data_path)[1:]
# train, val = create_train_test_split(df)
# _, _, x_val, y_val  = create_dataset(df, train,val)

time: 375 µs (started: 2023-11-28 11:05:55 +00:00)


In [9]:
# To load the model for inference
loaded_model = torch.load(path+"/model.pth")
loaded_model.eval()

SNN(
  (cnn1): Sequential(
    (0): Conv1d(1, 8, kernel_size=(3,), stride=(1,), padding=same)
    (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    (4): Conv1d(8, 12, kernel_size=(3,), stride=(1,), padding=same)
    (5): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  )
  (fc1): Sequential(
    (0): Linear(in_features=31344, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=1024, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)

time: 8.66 s (started: 2023-11-28 11:04:29 +00:00)


In [27]:
def predict_KNN(test_image,k=6):
  distances = []
  for i in range(len(x_train)):
    e1, e2 =loaded_model(torch.from_numpy(test_image).unsqueeze(1).to(device).float(),
                                  torch.from_numpy(x_train[i]).unsqueeze(1).to(device).float())
    eucledian_distance = F.pairwise_distance(e1, e2)
    distances.append(eucledian_distance[0].item())

  #k-nearest neighbors
  nearest_neighbors_indices = np.argsort(distances)[:k]
  print(y_train[nearest_neighbors_indices])

  #majority voting
  predicted_label = np.argmax(np.bincount(y_train[nearest_neighbors_indices]))
  return predicted_label

time: 943 µs (started: 2023-11-28 11:08:19 +00:00)


In [28]:
predict = []
for i in range(10):
  predict.append(predict_KNN(x_val[i]))
  # break

[0 1 0 2 0 1]
[0 1 0 0 0 1]
[0 1 2 2 0 0]
[1 2 1 1 1 1]
[2 2 0 2 0 0]
[0 0 0 0 1 1]
[1 0 0 0 2 2]
[1 1 1 1 1 1]
[0 0 0 0 1 0]
[2 2 2 2 2 0]
time: 1min 40s (started: 2023-11-28 11:08:22 +00:00)


In [29]:
print(predict)

[0, 0, 0, 1, 0, 0, 0, 1, 0, 2]
time: 5.36 ms (started: 2023-11-28 11:10:03 +00:00)


In [30]:
print(y_val[:10])

[1 2 0 1 0 2 0 2 1 1]
time: 1.35 ms (started: 2023-11-28 11:10:03 +00:00)


In [ ]:
while(True):
  pass